In [1]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import pandas as pd
from scipy import stats as st
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('video_games_sales.csv')
df

Rank                           Name                      basename  \
0          1                     Wii Sports                    wii-sports   
1          2              Super Mario Bros.              super-mario-bros   
2          3                 Mario Kart Wii                mario-kart-wii   
3          4  PlayerUnknown's Battlegrounds  playerunknowns-battlegrounds   
4          5              Wii Sports Resort             wii-sports-resort   
...      ...                            ...                           ...   
55787  55788                    Indivisible                   indivisible   
55788  55789                     Lost Ember                    lost-amber   
55789  55790                     Lost Ember                    lost-amber   
55790  55791                     Lost Ember                    lost-amber   
55791  55792                     Falcon Age                    falcon-age   

                  Genre ESRB_Rating Platform         Publisher  \
0                Sports           E      Wii          Nintendo   
1              Platform         NaN      NES          Nintendo   
2                Racing           E      Wii          Nintendo   
3               Shooter         NaN       PC  PUBG Corporation   
4                Sports           E      Wii          Nintendo   
...                 ...         ...      ...               ...   
55787      Role-Playing         NaN       PC         505 Games   
55788         Adventure          RP       PC   Mooneye Studios   
55789         Adventure          RP      PS4   Mooneye Studios   
55790         Adventure          RP     XOne   Mooneye Studios   
55791  Action-Adventure         NaN      PS4           Unknown   

              Developer  VGChartz_Score  Critic_Score  ...  NA_Sales  \
0          Nintendo EAD             NaN           7.7  ...       NaN   
1          Nintendo EAD             NaN          10.0  ...       NaN   
2          Nintendo EAD             NaN           8.2  ...       NaN   
3      PUBG Corporation             NaN           NaN  ...       NaN   
4          Nintendo EAD             NaN           8.0  ...       NaN   
...                 ...             ...           ...  ...       ...   
55787    Lab Zero Games             NaN           NaN  ...       NaN   
55788   Mooneye Studios             NaN           NaN  ...       NaN   
55789   Mooneye Studios             NaN           NaN  ...       NaN   
55790   Mooneye Studios             NaN           NaN  ...       NaN   
55791   Outerloop Games             NaN           NaN  ...       NaN   

       PAL_Sales  JP_Sales  Other_Sales    Year  Last_Update  \
0            NaN       NaN          NaN  2006.0          NaN   
1            NaN       NaN          NaN  1985.0          NaN   
2            NaN       NaN          NaN  2008.0  11th Apr 18   
3            NaN       NaN          NaN  2017.0  13th Nov 18   
4            NaN       NaN          NaN  2009.0          NaN   
...          ...       ...          ...     ...          ...   
55787        NaN       NaN          NaN  2019.0  11th Apr 19   
55788        NaN       NaN          NaN  2019.0  11th Apr 19   
55789        NaN       NaN          NaN  2019.0  11th Apr 19   
55790        NaN       NaN          NaN  2019.0  11th Apr 19   
55791        NaN       NaN          NaN     NaN  12th Apr 19   

                                                     url  status  \
0      http://www.vgchartz.com/game/2667/wii-sports/?...       1   
1      http://www.vgchartz.com/game/6455/super-mario-...       1   
2      http://www.vgchartz.com/game/6968/mario-kart-w...       1   
3      http://www.vgchartz.com/game/215988/playerunkn...       1   
4      http://www.vgchartz.com/game/24656/wii-sports-...       1   
...                                                  ...     ...   
55787  http://www.vgchartz.com/game/224682/indivisibl...       1   
55788  http://www.vgchartz.com/games/game.php?id=2246...       1   
55789  http://www.vgchartz.com/game/224690/l

In [3]:
# видим пропуски в оценках критиков - посчитаем сколько их 
df['Critic_Score'].isna().sum()

49256

Мы видим что пропусков слишком много - это значит, что их заполнение может привести к искажению данных - поэтому просто уберем их из данных для анализа 

In [4]:
df.dropna(subset=['Critic_Score'], inplace=True)
df

Rank                                  Name  \
0          1                            Wii Sports   
1          2                     Super Mario Bros.   
2          3                        Mario Kart Wii   
4          5                     Wii Sports Resort   
5          6    Pokemon Red / Green / Blue Version   
...      ...                                   ...   
55387  55388                   The Starship Damrey   
55423  55424                               Thumper   
55490  55491  Travis Strikes Again: No More Heroes   
55528  55529                            TumbleSeed   
55653  55654                     Wheels of Aurelia   

                                  basename             Genre ESRB_Rating  \
0                               wii-sports            Sports           E   
1                         super-mario-bros          Platform         NaN   
2                           mario-kart-wii            Racing           E   
4                        wii-sports-resort            Sports           E   
5                               pokmon-red      Role-Playing           E   
...                                    ...               ...         ...   
55387                  the-starship-damrey         Adventure         NaN   
55423                              thumper             Music         E10   
55490  travis-strikes-again-no-more-heroes            Action           M   
55528                           tumbleseed  Action-Adventure           E   
55653                    wheels-of-aurelia      Visual Novel           T   

      Platform                     Publisher                Developer  \
0          Wii                      Nintendo             Nintendo EAD   
1          NES                      Nintendo             Nintendo EAD   
2          Wii                      Nintendo             Nintendo EAD   
4          Wii                      Nintendo             Nintendo EAD   
5           GB                      Nintendo               Game Freak   
...        ...                           ...                      ...   
55387      3DS                       Level 5                  Level 5   
55423       NS                         Drool                    Drool   
55490       NS  Grasshopper Manufacture Inc.  Grasshopper Manufacture   
55528       NS                        aeiowu          Team TumbleSeed   
55653      PS4                  Mixedbag Srl        Santa Ragione Srl   

       VGChartz_Score  Critic_Score  ...  NA_Sales  PAL_Sales  JP_Sales  \
0                 NaN           7.7  ...       NaN        NaN       NaN   
1                 NaN          10.0  ...       NaN        NaN       NaN   
2                 NaN           8.2  ...       NaN        NaN       NaN   
4                 NaN           8.0  ...       NaN        NaN       NaN   
5                 NaN           9.4  ...       NaN        NaN       NaN   
...               ...           ...  ...       ...        ...       ...   
55387             NaN           5.4  ...       NaN        NaN       NaN   
55423             NaN           9.0  ...       NaN        NaN       NaN   
55490             NaN           8.0  ...       NaN        NaN       NaN   
55528             NaN           7.0  ...       NaN        NaN       NaN   
55653             NaN           8.0  ...       NaN        NaN       NaN   

       Other_Sales    Year  Last_Update  \
0              NaN  2006.0          NaN   
1              NaN  1985.0          NaN   
2              NaN  2008.0  11th Apr 18   
4              NaN  2009.0          NaN   
5              NaN  1998.0          NaN   
...            ...     ...          ...   
55387          NaN  2013.0  12th Apr 18   
55423          NaN  2017.0  20th Feb 18   
55490          NaN  2019.0  24th Mar 19   
55528          NaN  2017.0  11th Jan 18   
55653          NaN  2016.0  13th May 18   

                                                     url  status  \
0      http://www.vgchartz.com/game/2667/wii-sports/?...       1   
1      http://w

### Задание 1
#### Вернемся к набору данных о видеоиграх.

Ответьте на следующие вопросы:

1) Как критики относятся к спортивным играм?

2) Критикам нравятся больше игры на PC или на PS4?

3) Критикам больше нравятся стрелялки или стратегии?


#### Для каждого вопроса:

- сформулируйте нулевую и альтернативную гипотезы;
- выберите пороговый уровень статистической значимости;
- опишите полученные результаты статистического теста.

##### Как критики относятся к  спортивным видеоиграм
Нулевая гипотеза: Средняя оценка критиков по спортивным видеоиграм >= 7.0

In [5]:
sport_df = df.loc[df['Genre'] == 'Sports']
sport_df.head()

Rank               Name           basename   Genre ESRB_Rating Platform  \
0      1         Wii Sports         wii-sports  Sports           E      Wii   
4      5  Wii Sports Resort  wii-sports-resort  Sports           E      Wii   
16    17            Wii Fit            wii-fit  Sports           E      Wii   
17    18       Wii Fit Plus       wii-fit-plus  Sports           E      Wii   
68    69            FIFA 18            fifa-18  Sports           E      PS4   

    Publisher     Developer  VGChartz_Score  Critic_Score  ...  NA_Sales  \
0    Nintendo  Nintendo EAD             NaN           7.7  ...       NaN   
4    Nintendo  Nintendo EAD             NaN           8.0  ...       NaN   
16   Nintendo  Nintendo EAD             NaN           7.9  ...       NaN   
17   Nintendo  Nintendo EAD             NaN           8.0  ...       NaN   
68  EA Sports  EA Vancouver             NaN           8.3  ...      1.27   

    PAL_Sales  JP_Sales  Other_Sales    Year  Last_Update  \
0         NaN       NaN          NaN  2006.0          NaN   
4         NaN       NaN          NaN  2009.0          NaN   
16        NaN       NaN          NaN  2008.0          NaN   
17        NaN       NaN          NaN  2009.0          NaN   
68       8.64      0.15         1.73  2017.0  02nd Apr 18   

                                                  url  status Vgchartzscore  \
0   http://www.vgchartz.com/game/2667/wii-sports/?...       1           NaN   
4   http://www.vgchartz.com/game/24656/wii-sports-...       1           8.8   
16  http://www.vgchartz.com/game/7480/wii-fit/?reg...       1           8.0   
17  http://www.vgchartz.com/game/35077/wii-fit-plu...       1           NaN   
68  http://www.vgchartz.com/game/180525/fifa-18/?r...       1           NaN   

                                          img_url  
0   /games/boxart/full_2258645AmericaFrontccc.jpg  
4   /games/boxart/full_7295041AmericaFrontccc.jpg  
16  /games/boxart/full_3619557AmericaFrontccc.jpg  
17  /games/boxart/full_2716475AmericaFrontccc.jpg  
68  /games/boxart/full_5257064AmericaFrontccc.jpg  

[5 rows x 23 columns]

In [6]:
alpha = 0.05
stat, p = st.ttest_1samp(sport_df['Critic_Score'], 7.0)
print('Statistics=%.3f, p=%.3f' % (stat, p))

if (stat < 0) & (p / 2 < alpha):
    print('Отвергаем нулевую гипотезу, критикам не нравятся спортивные видеоигры')
else:
    print('Не отвергаем нулевую нулевую гипотезу, критикам нравятся спортивные видеоигры')

Statistics=5.197, p=0.000
Не отвергаем нулевую нулевую гипотезу, критикам нравятся спортивные видеоигры


##### Критикам нравятся больше игры на PC или на PS4?

Нулевая гипотеза: критикам одинаково нравятся игры на PC и на PS4

In [7]:
pc_df = df.loc[df['Platform'] == 'PC']
ps4_df = df.loc[df['Platform'] == 'PS4']

In [8]:
alpha = 0.05
stat, p = st.ttest_ind(pc_df['Critic_Score'], ps4_df['Critic_Score'], equal_var=False)
print('Statistics=%.3f, p=%.3f' % (stat, p))

if (p < alpha):
    print('Отвергаем нулевую гипотезу, есть разница в отношении критиков к играм на PC и на PS4')
else:
    print('Не отвергаем нулевую гипотезу, критикам одинаково нравятся игры на PC и на PS4')

Statistics=-2.739, p=0.007
Отвергаем нулевую гипотезу, есть разница в отношении критиков к играм на PC и на PS4


По результатам теста мы можем отвергнуть нулевую гипотезу и увидеть, что значение t-критерия отрицательно - значит критики более высоко оценивают игры на платформе PS4

##### Критикам больше нравятся стрелялки или стратегии?
Нулевая гипотеза: стрелялки и стратегии критики в среднем оценивают одинаково

In [9]:
shooters = df.loc[df['Genre'] == 'Shooter']
strategies = df.loc[df['Genre'] == 'Strategy']

In [10]:
alpha = 0.05
stat, p = st.ttest_ind(shooters['Critic_Score'], strategies['Critic_Score'], equal_var=False)
print('Statistics=%.3f, p=%.3f' % (stat, p))

if (p < alpha):
    print('Отвергаем нулевую гипотезу, есть разница в отношении критиков к стрелялкам и стратегиям')
else:
    print('Не отвергаем нулевую гипотезу, критики одинаково оценивают стрелялки и стратегии')

Statistics=-1.607, p=0.108
Не отвергаем нулевую гипотезу, критики одинаково оценивают стрелялки и стратегии


По результатам теста мы не можем отвергнуть нулевую гипотезу - отсутствует статистически значимая разница между оценкой стрелялок и стратегий. 
Но мы можем увидеть, благодаря отрицательному значению t-критерия, что критики более высоко оценивают стратегии

### Задание 2

Реализуйте базовую модель логистической регрессии для классификации текстовых сообщений по признаку спама. 

Для этого:

1) Приведите весь текст к нижнему регистру

2) Удалите мусорные символы

3) Удалите стоп-слова

4) Приведите все слова к нормальной форме

5) Преобразуйте все сообщения в вектора TF-IDF. Вам поможет следующий код:

```
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df.Message)
names = tfidf.get_feature_names()
tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=cname)
```

Можете поэкспериментировать с параметрами TfidfVectorizer.

6) Разделите данные на тестовые и тренировочные в соотношении 30/70, укажите random_state=42. Используйте train_test_split

7) Постройте модель логистической регрессии, укажите random_state=42, оцените ее точность на тестовых данных

8) Опишите результаты при помощи confusion_matrix

9) Постройте датафрейм, который будет содержать все исходные тексты сообщений, классифицированные неправильно (с указанием фактического и предсказанного).

In [11]:
main_df = pd.read_csv('spam.csv')
main_df

Category                                            Message
0         ham  Go until jurong point, crazy.. Available only ...
1         ham                      Ok lar... Joking wif u oni...
2        spam  Free entry in 2 a wkly comp to win FA Cup fina...
3         ham  U dun say so early hor... U c already then say...
4         ham  Nah I don't think he goes to usf, he lives aro...
...       ...                                                ...
5567     spam  This is the 2nd time we have tried 2 contact u...
5568      ham               Will ü b going to esplanade fr home?
5569      ham  Pity, * was in mood for that. So...any other s...
5570      ham  The guy did some bitching but I acted like i'd...
5571      ham                         Rofl. Its true to its name

[5572 rows x 2 columns]

In [13]:
# по условию в итоговый датафрейм должны попасть исходные тексты - работать будем с копией датафрейма
df = main_df.copy(deep=True)

##### Форматируем текст

In [14]:
df["Message"] = df['Message'].str.lower().replace('[\W_]+',' ', regex = True)

##### Удаляем стоп-слова

In [15]:
stopwords_set = set(stopwords.words('english'))
df['Message'] = df['Message'].apply(lambda message: [word for word in message.split() if word not in stopwords_set])

##### Лемматизируем слова 

In [16]:
wordnet_lemmatizer = WordNetLemmatizer()
df['Message'] = df['Message'].apply(lambda message: " ".join([wordnet_lemmatizer.lemmatize(word) for word in message]))

##### Приводим сообщения  к векторам 

In [17]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df.Message)
names = tfidf.get_feature_names()
tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=names)

In [18]:
tfidf_matrix.head(20)

00       000  000pes  008704050406  0089  0121  01223585236  01223585334  \
0   0.0  0.000000     0.0           0.0   0.0   0.0          0.0          0.0   
1   0.0  0.000000     0.0           0.0   0.0   0.0          0.0          0.0   
2   0.0  0.000000     0.0           0.0   0.0   0.0          0.0          0.0   
3   0.0  0.000000     0.0           0.0   0.0   0.0          0.0          0.0   
4   0.0  0.000000     0.0           0.0   0.0   0.0          0.0          0.0   
5   0.0  0.000000     0.0           0.0   0.0   0.0          0.0          0.0   
6   0.0  0.000000     0.0           0.0   0.0   0.0          0.0          0.0   
7   0.0  0.000000     0.0           0.0   0.0   0.0          0.0          0.0   
8   0.0  0.000000     0.0           0.0   0.0   0.0          0.0          0.0   
9   0.0  0.000000     0.0           0.0   0.0   0.0          0.0          0.0   
10  0.0  0.000000     0.0           0.0   0.0   0.0          0.0          0.0   
11  0.0  0.208044     0.0           0.0   0.0   0.0          0.0          0.0   
12  0.0  0.224267     0.0           0.0   0.0   0.0          0.0          0.0   
13  0.0  0.000000     0.0           0.0   0.0   0.0          0.0          0.0   
14  0.0  0.000000     0.0           0.0   0.0   0.0          0.0          0.0   
15  0.0  0.000000     0.0           0.0   0.0   0.0          0.0          0.0   
16  0.0  0.000000     0.0           0.0   0.0   0.0          0.0          0.0   
17  0.0  0.000000     0.0           0.0   0.0   0.0          0.0          0.0   
18  0.0  0.000000     0.0           0.0   0.0   0.0          0.0          0.0   
19  0.0  0.000000     0.0           0.0   0.0   0.0          0.0          0.0   

    0125698789   02  ...  zhong  zindgi  zoe  zogtorius  zoom  zouk  zyada  \
0          0.0  0.0  ...    0.0     0.0  0.0        0.0   0.0   0.0    0.0   
1          0.0  0.0  ...    0.0     0.0  0.0        0.0   0.0   0.0    0.0   
2          0.0  0.0  ...    0.0     0.0  0.0        0.0   0.0   0.0    0.0   
3          0.0  0.0  ...    0.0     0.0  0.0        0.0   0.0   0.0    0.0   
4          0.0  0.0  ...    0.0     0.0  0.0        0.0   0.0   0.0    0.0   
5          0.0  0.0  ...    0.0     0.0  0.0        0.0   0.0   0.0    0.0   
6          0.0  0.0  ...    0.0     0.0  0.0        0.0   0.0   0.0    0.0   
7          0.0  0.0  ...    0.0     0.0  0.0        0.0   0.0   0.0    0.0   
8          0.0  0.0  ...    0.0     0.0  0.0        0.0   0.0   0.0    0.0   
9          0.0  0.0  ...    0.0     0.0  0.0        0.0   0.0   0.0    0.0   
10         0.0  0.0  ...    0.0     0.0  0.0        0.0   0.0   0.0    0.0   
11         0.0  0.0  ...    0.0     0.0  0.0        0.0   0.0   0.0    0.0   
12         0.0  0.0  ...    0.0     0.0  0.0        0.0   0.0   0.0    0.0   
13         0.0  0.0  ...    0.0     0.0  0.0        0.0   0.0   0.0    0.0   
14         0.0  0.0  ...    0.0     0.0  0.0        0.0   0.0   0.0    0.0   
15         0.0  0.0  ...    0.0     0.0  0.0        0.0   0.0   0.0    0.0   
16         0.0  0.0  ...    0.0     0.0  0.0        0.0   0.0   0.0    0.0   
17         0.0  0.0  ...    0.0     0.0  0.0        0.0   0.0   0.0    0.0   
18         0.0  0.0  ...    0.0     0.0  0.0        0.0   0.0   0.0    0.0   
19         0.0  0.0  ...    0.0     0.0  0.0        0.0   0.0   0.0    0.0   

     èn        ú1  〨ud  
0   0.0  0.000000  0.0  
1   0.0  0.000000  0.0  
2   0.0  0.000000  0.0  
3   0.0  0.000000  0.0  
4   0.0  0.000000  0.0  
5   0.0  0.000000  0.0  
6   0.0  0.000000  0.0  
7   0.0  0.000000  0.0  
8   0.0  0.000000  0.0  
9   0.0  0.000000  0.0  
10  0.0  0.000000  0.0  
11  0.0  0.000000  0.0  
12  0.0  0.000000  0.0  
13  0.0  0.000000  0.0  
14  0.0  0.000000  0.0  
15  0.0  0.000000  0.0  
16  0.0  0.000000  0.0  
17  0.0  0.000000  0.0  
18  0.0  0.000000  0.0  
19  0.0  0.227922  0.0  

[20 rows x 8048 columns]

##### Делим данные на тестовые и тренировочные

In [19]:
tfidf_matrix["Category"] = df["Category"]
data_train, data_test = train_test_split(tfidf_matrix, test_size=0.3, random_state=42)
y_train, y_test = data_train["Category"], data_test["Category"]
del data_train["Category"]
del data_test["Category"]
del tfidf_matrix["Category"]

##### Строим модель логистической регрессии

In [20]:
lda = LogisticRegression(random_state=42)
lda.fit(data_train, y_train)

# проверяем точность регрессии
accuracy_score(y_test, lda.predict(data_test))

0.958732057416268

Точность довольно высокая - 95%

##### Описываем результат с помощью confusion_matrix

In [21]:
full_predict = lda.predict(tfidf_matrix)

confusion_matrix(df["Category"], full_predict)

array([[4817,    8],
       [ 197,  550]])

На основной диагонали лежат верно определенные по категориям сообщения, на второстепенной диагонали - ошибочно определенные

Первая строка - сообщения определенные как спам, вторая строка - сообщения, определенные как не спам

##### Строим датафрейм с текстами сообщений, которые были классифицированы неправильно

In [24]:
predict_df = pd.DataFrame(full_predict)
wrong_df = predict_df.join(main_df, how="left")
wrong_df.rename(columns={0: 'Predict'}, inplace=True)
wrong_df = wrong_df.loc[wrong_df['Predict'] != wrong_df['Category']]

In [26]:
# результирующий датафрейм
wrong_df.head()

Predict Category                                            Message
5      ham     spam  FreeMsg Hey there darling it's been 3 week's n...
15     ham     spam  XXXMobileMovieClub: To use your credit, click ...
19     ham     spam  England v Macedonia - dont miss the goals/team...
54     ham     spam  SMS. ac Sptv: The New Jersey Devils and the De...
56     ham     spam  Congrats! 1 year special cinema pass for 2 is ...

Количество строк датафрейма и сумма чисел второстепенной диагонали совпадают 197 + 8 = 205 - ничего не потерялось